```{julia}
projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "CounterfactualTraining.jl" for s in ss])[1]]...) 
cd(projectdir)

using CTExperiments
using CTExperiments.CSV
using CTExperiments.DataFrames
using CTExperiments.Plots
using CTExperiments.StatsBase
using Random

using DotEnv
DotEnv.load!()
```

```{julia}
res_dir = joinpath(ENV["FINAL_GRID_RESULTS"],"single")
mtbl_dir = joinpath(ENV["FINAL_GRID_RESULTS"],"mutability")
```


# Details on Main Experiments {#sec-app-main .appendix}

## Final Hyperparameters

As discussed @sec-experiments, CT is sensitive to certain hyperparameter choices. We study the effect of many hyperparameters extensively in @sec-app-grid. For the main results, we tune a small set of key hyperparameters (@sec-app-tune). The final choices for the main results are presented for each data set in @tbl-final-params along with training, test and batch sizes.

::: {#tbl-final-params}

::: {.content-hidden unless-format="pdf"}


```{julia}
#| output: asis

df = final_params(res_dir)
get_table_inputs(df, nothing; backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; table_type=:tabular)
```


:::

Final hyperparameters used for the main results for the different datasets.

:::


```{julia}
#| eval: false

tbl_mtbl = final_mutability(mtbl_dir; var=["distance"])
df = final_table(res_dir; tbl_mtbl)
inputs = get_table_inputs(df, nothing; backend=Val(:latex))
tabulate_results(inputs; table_type=:tabular, save_name="paper/quarto_ecml/tables/main.tex", wrap_table=false)
tabulate_results(inputs; table_type=:tabular, save_name="paper/preprint/tables/main.tex", wrap_table=false)
```


## Qualitative Findings for Image Data


```{julia}
#| eval: false

Random.seed!(42)    # change seed for different outcome
overwrite = false   # set to `true` if you want to overwrite the file
fname = joinpath(res_dir,"mlp/mnist/grid_config.toml")
output_dir = mkpath("paper/experiments/output/extra/")
gen = CTExperiments.GeneratorParams(lambda_energy=25.0)
conv = "max_iter"
cfg = EvaluationConfig(
    grid_file=fname, 
    counterfactual_params=(conv=conv, generator_params=gen), 
    save_dir=output_dir,
    test_time=true
)
generate_factual_target_pairs(cfg; overwrite, nce=10)
plts = plot_ce(MNIST(), cfg; byvars="objective")
savefig(plts["objective"]["full"]["ecco"], "paper/figures/mnist_mlp.png")
savefig(plts["objective"]["vanilla"]["ecco"], "paper/figures/mnist_mlp_vanilla.png")
```




@fig-mnist shows much more plausible (faithful) counterfactuals for a model with CT than the model with conventional training (@fig-mnist-vanilla).

::: {layout="[10,-2,10]" layout-valign="top"}
![Counterfactual images for *MLP* with counterfactual training. Factual images are shown on the diagonal, with the corresponding counterfactual for each target class (columns) in that same row. The underlying generator, *ECCo*, aims to generate counterfactuals that are faithful to the model [@altmeyer2024faithful].](/paper/figures/mnist_mlp.png){#fig-mnist}

![The same setup, factuals, model architecture and generator as in @fig-mnist, but the model was trained with CT.](/paper/figures/mnist_mlp_vanilla.png){#fig-mnist-vanilla}
:::